In [16]:
import os
import shutil
import numpy as np
import pandas as pd
# from tqdm.notebook import tqdm

In [17]:
BASE_DIR = ''
COUNTRY = 'malawi_2016'
RANDOM_SEED = 7 # for reproducibility
COUNTRY_DIR = os.path.join(BASE_DIR, 'countries', COUNTRY)
PROCESSED_DIR = os.path.join(COUNTRY_DIR, 'processed')

# these relate to the current country in question
IMAGE_DIR = os.path.join(COUNTRY_DIR, 'images')

# these relate to training the CNN to predict nightlights
CNN_TRAIN_IMAGE_DIR = os.path.join(BASE_DIR, 'data/cnn_images')
CNN_SAVE_DIR = os.path.join(BASE_DIR, 'models')


In [18]:
os.makedirs(CNN_TRAIN_IMAGE_DIR, exist_ok=False)
os.makedirs(CNN_SAVE_DIR, exist_ok=True)

FileExistsError: [Errno 17] File exists: 'data/cnn_images'

# Preprocess

In [11]:
df_download = pd.read_csv(os.path.join(PROCESSED_DIR, 'image_download_locs.csv'))
downloaded = os.listdir(IMAGE_DIR)
assert len(df_download.set_index('image_name').drop(downloaded)) == 0, print('Not all unique images have been downloaded')


In [12]:
df_download.head()

,im_lat,im_lon,cluster_lat,cluster_lon,cons_pc,nightlights,nightlights_bin,image_name
0,-17.075000,35.183332,-17.09515,35.217213,1.477796,0.0,1,-17.07499963170001_35.1833324726.png
1,-17.058333,35.191666,-17.09515,35.217213,1.477796,0.0,1,-17.058332965100007_35.191665805900016.png
2,-17.075000,35.191666,-17.09515,35.217213,1.477796,0.0,1,-17.07499963170001_35.191665805900016.png
3,-17.125000,35.199999,-17.09515,35.217213,1.477796,0.0,1,-17.124999631500014_35.1999991392.png
4,-17.075000,35.208332,-17.09515,35.217213,1.477796,0.0,1,-17.07499963170001_35.208332472500025.png


In [13]:
# the distribution
(df_download['nightlights_bin']==1).mean(), (df_download['nightlights_bin']==2).mean(), (df_download['nightlights_bin']==3).mean()


(0.51321986726611, 0.2338096767287519, 0.25297045600513807)

Creates train/valid folders

The model will use these directly.

In [14]:
# we need this renaming because images can have the same name when they belong to different clusters
# this makes unique image names, even though some are really the same image
df_download['image_renamed'] = df_download['cluster_lat'].astype(str) + '_' + df_download['cluster_lon'].astype(str) + '_' + df_download['image_name']


In [15]:
df_download.head()

,im_lat,im_lon,cluster_lat,cluster_lon,cons_pc,nightlights,nightlights_bin,image_name,image_renamed
0,-17.075000,35.183332,-17.09515,35.217213,1.477796,0.0,1,-17.07499963170001_35.1833324726.png,-17.09515_35.217213_-17.07499963170001_35.1833...
1,-17.058333,35.191666,-17.09515,35.217213,1.477796,0.0,1,-17.058332965100007_35.191665805900016.png,-17.09515_35.217213_-17.058332965100007_35.191...
2,-17.075000,35.191666,-17.09515,35.217213,1.477796,0.0,1,-17.07499963170001_35.191665805900016.png,-17.09515_35.217213_-17.07499963170001_35.1916...
3,-17.125000,35.199999,-17.09515,35.217213,1.477796,0.0,1,-17.124999631500014_35.1999991392.png,-17.09515_35.217213_-17.124999631500014_35.199...
4,-17.075000,35.208332,-17.09515,35.217213,1.477796,0.0,1,-17.07499963170001_35.208332472500025.png,-17.09515_35.217213_-17.07499963170001_35.2083...


In [16]:
pic_list = df_download['image_renamed'].values.tolist()
to_pick = int(0.8*len(pic_list)); to_pick

29894

In [17]:
np.random.seed(RANDOM_SEED)
inds = np.arange(len(pic_list))
train_ind = np.random.choice(np.arange(len(pic_list)), to_pick, replace=False)
valid_ind = np.delete(inds, train_ind)

In [18]:
pic_list = np.array(pic_list)
train_im = pic_list[train_ind]
valid_im = pic_list[valid_ind]

In [19]:
os.makedirs(os.path.join(CNN_TRAIN_IMAGE_DIR, 'train'), exist_ok=False)
os.makedirs(os.path.join(CNN_TRAIN_IMAGE_DIR, 'valid'), exist_ok=False)

labels = ['1', '2', '3']
for l in labels:
    os.makedirs(os.path.join(CNN_TRAIN_IMAGE_DIR, 'train', l), exist_ok=False)
    os.makedirs(os.path.join(CNN_TRAIN_IMAGE_DIR, 'valid', l), exist_ok=False)

In [20]:
t = df_download.iloc[train_ind]
v = df_download.iloc[valid_ind]

In [21]:
len(t), len(v)

(29894, 7474)

In [22]:
# copies images to their respective folders
# YES this does mean an image can be in train and valid, but that's ok
# because we do not really care about accuracy on nightlights
print('copying train images')
for im_name, im_renamed, l in tqdm(zip(t['image_name'], t['image_renamed'], t['nightlights_bin']), total=len(t)):
    shutil.copy(os.path.join(IMAGE_DIR, im_name), os.path.join(CNN_TRAIN_IMAGE_DIR, 'train', str(l), im_renamed))

copying train images



copying valid images


OSError: [Errno 28] No space left on device

In [23]:
# Do the same for validation set
print('copying valid images')
for im_name, im_renamed, l in tqdm(zip(v['image_name'], v['image_renamed'], v['nightlights_bin']), total=len(v)):
    shutil.copy(os.path.join(IMAGE_DIR, im_name), os.path.join(CNN_TRAIN_IMAGE_DIR, 'valid', str(l), im_renamed))

copying valid images


In [19]:
# shows count distribution in train folder, make sure this matches above
counts = []
for l in ['1', '2', '3']:
    counts.append(len(os.listdir(os.path.join(CNN_TRAIN_IMAGE_DIR, 'train', l))))
print(counts)
print([c/sum(counts) for c in counts])
print(sum(counts))

[15365, 7006, 7523]
[0.5139827390111729, 0.23436141031645147, 0.2516558506723757]
29894


In [20]:
# shows count distribution in valid folder
counts = []
for l in ['1', '2', '3']:
    counts.append(len(os.listdir(os.path.join(CNN_TRAIN_IMAGE_DIR, 'valid', l))))
print(counts)
print([c/sum(counts) for c in counts])
print(sum(counts))

[3814, 1732, 1931]
[0.5100976327404039, 0.23164370736926576, 0.25825865989033037]
7477


# Train Model
Heavily adapted from the PyTorch CNN training tutorial.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [27]:
torch.cuda.current_device()

AssertionError: Torch not compiled with CUDA enabled

In [22]:
# Top level data directory.

data_dir = CNN_TRAIN_IMAGE_DIR

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = "vgg"

# Number of classes in the dataset
num_classes = 3

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for
num_epochs = 5

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

In [23]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224
    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [24]:
# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
model_ft

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(ke

In [25]:
# you can modify the classifier part of the model by doing this
# model_ft.classifier = nn.Sequential(
#     nn.Linear(in_features=25088, out_features=4096, bias=True),
#     nn.ReLU(inplace=True),
#     nn.Dropout(p=0.5),
#     nn.Linear(in_features=4096, out_features=256, bias=True),
#     nn.ReLU(inplace=True),
#     nn.Dropout(p=0.5),
#     nn.Linear(in_features=256, out_features=3, bias=True),
# )

In [26]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'valid']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'valid']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device:', device)

Initializing Datasets and Dataloaders...
device: cpu


In [36]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=1e-4, momentum=0.9)

Params to learn:
	 classifier.6.weight
	 classifier.6.bias


In [34]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'valid':
                val_acc_history.append(epoch_acc)
                
        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [35]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs)

NameError: name 'optimizer_ft' is not defined

In [26]:
path = os.path.join(CNN_SAVE_DIR, 'trained_model.pt')
assert not os.path.isfile(path), print('A model is already saved at this location')
print(f'Saving model to {path}')
torch.save(model_ft, path)

Saving model to ../models/trained_model.pt


In [27]:
# you can run below if you want to see the final accuracy on nightlights over the train set
model_ft.eval()   # Set model to evaluate mode

criterion = nn.CrossEntropyLoss()
running_loss = 0.0
running_corrects = 0
total = 0

# Iterate over data.
for inputs, labels in tqdm(dataloaders_dict['train']):
    inputs = inputs.to(device)
    labels = labels.to(device)

    # forward
    # track history if only in train
    with torch.set_grad_enabled(False):
        outputs = model_ft(inputs)
        loss = criterion(outputs, labels)

        _, preds = torch.max(outputs, 1)

    # statistics
    running_loss += loss.item() * inputs.size(0)
    running_corrects += torch.sum(preds == labels.data)
    
    total += len(preds)
        
print(running_corrects.double()/total)


tensor(0.8153, device='cuda:0', dtype=torch.float64)
